
## 快速开始


首先安装依赖项：

```
pip install -r requirement.txt
```

第一步：数据处理：
该数据集就位于项目目录下的`data`文件夹里。


In [2]:
from os.path import join
from codecs import open
import pickle
import random
def build_map(lists):
    #maps = {}
    # 该函数用于构建词表,得到word和id对应的字典类型，请同学们完成本函数的构建
    pass


def build_corpus(split, make_vocab=True):
    """读取数据"""
    with open(join('data/'+split+'Content.pkl'), 'rb') as fContent:
        word_lists = pickle.load(fContent)
    with open(join('data/'+split+'Tag.pkl'), 'rb') as fTag:
        tag_lists = pickle.load(fTag)
    for i, item in enumerate(word_lists):
        word_lists[i] = list(item)
    if make_vocab:
        word2id = build_map(word_lists)
        tag2id = build_map(tag_lists)
        print('vocab构建成功！')
        return word_lists, tag_lists, word2id, tag2id
    else:
        return word_lists, tag_lists

print("读取数据...")   # 这里只提供训练数据和测试数据
train_word_lists_full, train_tag_lists_full, word2id, tag2id = build_corpus("train")  # 获取训练数据输入、标签、word2id、tag2id
test_word_lists_full, test_tag_lists_full = build_corpus("test", make_vocab=False)  # 获得测数数据的
print(len(train_word_lists_full))   # 训练数据条数
print(len(test_word_lists_full)) # 测数数据条数
print(train_word_lists_full[0])  # 训练数据例子
print(train_tag_lists_full[0])  # 训练标签例子
'''
由于使用完整数据训练模型速度较慢，这里采用随机采样方式采样出部分训练和测试数据！
大家如果有兴趣可以自己用全部数据训练模型哦
'''
# 随机采样的方式选出规定数目的训练和测试数据
train_select_num = 500  # 采样训练数据的数目
test_select_num = 50  # 采样测试数据的数目
train_select_idx = random.sample([i for i in range(len(train_word_lists_full))], train_select_num)
test_select_idx = random.sample([i for i in range(len(test_word_lists_full))], test_select_num)
train_word_lists = list([train_word_lists_full[idx] for idx in train_select_idx])
train_tag_lists = list([train_tag_lists_full[idx] for idx in train_select_idx])
test_word_lists = list([test_word_lists_full[idx] for idx in test_select_idx])
test_tag_lists = list([test_tag_lists_full[idx] for idx in test_select_idx])

## 隐马尔可夫模型（Hidden Markov Model，HMM）	 

隐马尔可夫模型描述由一个隐藏的马尔科夫链随机生成不可观测的状态随机序列，再由各个状态生成一个观测而产生观测随机序列的过程（李航 统计学习方法）。隐马尔可夫模型由初始状态分布，状态转移概率矩阵以及观测概率矩阵所确定。

命名实体识别本质上可以看成是一种序列标注问题，在使用HMM解决命名实体识别这种序列标注问题的时候，我们所能观测到的是字组成的序列（观测序列），观测不到的是每个字对应的标注（状态序列）。

**初始状态分布**就是每一个标注的初始化概率，**状态转移概率矩阵**就是由某一个标注转移到下一个标注的概率（就是若前一个词的标注为$tag_i$ ，则下一个词的标注为$tag_j$的概率为 $M_{ij}$），**观测概率矩阵**就是指在某个标注下，生成某个词的概率。

HMM模型的训练过程对应隐马尔可夫模型的学习问题（李航 统计学习方法），

实际上就是根据训练数据根据最大似然的方法估计模型的三个要素，即上文提到的初始状态分布、状态转移概率矩阵以及观测概率矩阵，模型训练完毕之后，利用模型进行解码，即对给定观测序列，求它对应的状态序列，这里就是对给定的句子，求句子中的每个字对应的标注，针对这个解码问题，我们使用的是维特比（viterbi）算法。


In [3]:
import torch
class HMM(object):
    def __init__(self, N, M):
        """Args:
            N: 状态数，这里对应存在的标注的种类
            M: 观测数，这里对应有多少不同的字
        """
        self.N = N
        self.M = M

        # 状态转移概率矩阵 A[i][j]表示从i状态转移到j状态的概率
        self.A = torch.zeros(N, N)
        # 观测概率矩阵, B[i][j]表示i状态下生成j观测的概率
        self.B = torch.zeros(N, M)
        # 初始状态概率  Pi[i]表示初始时刻为状态i的概率
        self.Pi = torch.zeros(N)

    def train(self, word_lists, tag_lists, word2id, tag2id):
        """请大家完成马尔可夫模型概率转移代码
        参数:
            word_lists: 列表，其中每个元素由字组成的列表，如 ['担','任','科','员']
            tag_lists: 列表，其中每个元素是由对应的标注组成的列表，如 ['O','O','B-TITLE', 'E-TITLE']
            word2id: 将字映射为ID
            tag2id: 字典，将标注映射为ID
        """
        assert len(tag_lists) == len(word_lists)
        # 估计转移概率矩阵
        
        # 估计观测概率矩阵

        # 估计初始状态概率


    def test(self, word_lists, word2id, tag2id):
        # 请大家完成test函数，得到预测的标签列表。生成测试标签需要用到维特比算法，我们给出该算法的完整代码，self.decoding函数。
        pred_tag_lists = []
        return pred_tag_lists

    def decoding(self, word_list, word2id, tag2id):
        """
        使用维特比算法对给定观测序列求状态序列， 这里就是对字组成的序列,求其对应的标注。
        维特比算法实际是用动态规划解隐马尔可夫模型预测问题，即用动态规划求概率最大路径（最优路径）
        这时一条路径对应着一个状态序列
        """
        # 问题:整条链很长的情况下，十分多的小概率相乘，最后可能造成下溢
        # 解决办法：采用对数概率，这样源空间中的很小概率，就被映射到对数空间的大的负数
        #  同时相乘操作也变成简单的相加操作
        A = torch.log(self.A)
        B = torch.log(self.B)
        Pi = torch.log(self.Pi)

        # 初始化 维比特矩阵viterbi 它的维度为[状态数, 序列长度]
        # 其中viterbi[i, j]表示标注序列的第j个标注为i的所有单个序列(i_1, i_2, ..i_j)出现的概率最大值
        seq_len = len(word_list)
        viterbi = torch.zeros(self.N, seq_len)
        # backpointer是跟viterbi一样大小的矩阵
        # backpointer[i, j]存储的是 标注序列的第j个标注为i时，第j-1个标注的id
        # 等解码的时候，我们用backpointer进行回溯，以求出最优路径
        backpointer = torch.zeros(self.N, seq_len).long()

        # self.Pi[i] 表示第一个字的标记为i的概率
        # Bt[word_id]表示字为word_id的时候，对应各个标记的概率
        # self.A.t()[tag_id]表示各个状态转移到tag_id对应的概率

        # 所以第一步为
        start_wordid = word2id.get(word_list[0], None)
        Bt = B.t()
        if start_wordid is None:
            # 如果字不再字典里，则假设状态的概率分布是均匀的
            bt = torch.log(torch.ones(self.N) / self.N)
        else:
            bt = Bt[start_wordid]
        viterbi[:, 0] = Pi + bt
        backpointer[:, 0] = -1

        # 递推公式：
        # viterbi[tag_id, step] = max(viterbi[:, step-1]* self.A.t()[tag_id] * Bt[word])
        # 其中word是step时刻对应的字
        # 由上述递推公式求后续各步
        for step in range(1, seq_len):
            wordid = word2id.get(word_list[step], None)
            # 处理字不在字典中的情况
            # bt是在t时刻字为wordid时，状态的概率分布
            if wordid is None:
                # 如果字不再字典里，则假设状态的概率分布是均匀的
                bt = torch.log(torch.ones(self.N) / self.N)
            else:
                bt = Bt[wordid]  # 否则从观测概率矩阵中取bt
            for tag_id in range(len(tag2id)):
                max_prob, max_id = torch.max(
                    viterbi[:, step-1] + A[:, tag_id],
                    dim=0
                )
                viterbi[tag_id, step] = max_prob + bt[tag_id]
                backpointer[tag_id, step] = max_id

        # 终止， t=seq_len 即 viterbi[:, seq_len]中的最大概率，就是最优路径的概率
        best_path_prob, best_path_pointer = torch.max(
            viterbi[:, seq_len-1], dim=0
        )

        # 回溯，求最优路径
        best_path_pointer = best_path_pointer.item()
        best_path = [best_path_pointer]
        for back_step in range(seq_len-1, 0, -1):
            best_path_pointer = backpointer[best_path_pointer, back_step]
            best_path_pointer = best_path_pointer.item()
            best_path.append(best_path_pointer)

        # 将tag_id组成的序列转化为tag
        assert len(best_path) == len(word_list)
        id2tag = dict((id_, tag) for tag, id_ in tag2id.items())
        tag_list = [id2tag[id_] for id_ in reversed(best_path)]

        return tag_list

In [4]:
import time
from collections import Counter
import pickle
from utils import save_model, flatten_lists, load_model
from evaluating import Metrics
import torch

def hmm_train_eval(train_data, test_data, word2id, tag2id, remove_O=False):
    """训练并评估hmm模型"""
    # 训练HMM模型
    train_word_lists, train_tag_lists = train_data
    test_word_lists, test_tag_lists = test_data

    hmm_model = HMM(len(tag2id), len(word2id))
    hmm_model.train(train_word_lists,
                    train_tag_lists,
                    word2id,
                    tag2id)
    save_model(hmm_model, "./ckpts/hmm.pkl")

    # 评估hmm模型
    pred_tag_lists = hmm_model.test(test_word_lists,
                                    word2id,
                                    tag2id)

    metrics = Metrics(test_tag_lists, pred_tag_lists, remove_O=remove_O)
    metrics.report_scores()
    metrics.report_confusion_matrix()

    return pred_tag_lists

# 训练评估hmm模型
print("正在训练评估HMM模型...")
hmm_pred = hmm_train_eval(
    (train_word_lists, train_tag_lists),
    (test_word_lists, test_tag_lists),
    word2id,
    tag2id
)

## 条件随机场（Conditional Random Field, CRF)

 HMM模型中存在两个假设，一是输出观察值之间严格独立，二是状态转移过程中当前状态只与前一状态有关。也就是说，在命名实体识别的场景下，HMM认为观测到的句子中的每个字都是相互独立的，而且当前时刻的标注只与前一时刻的标注相关。但实际上，命名实体识别往往需要更多的特征，比如词性，词的上下文等等，同时当前时刻的标注应该与前一时刻以及后一时刻的标注都相关联。由于这两个假设的存在，显然HMM模型在解决命名实体识别的问题上是存在缺陷的。

条件随机场通过引入自定义的特征函数，不仅可以表达观测之间的依赖，还可表示当前观测与前后多个状态之间的复杂依赖，可以有效克服HMM模型面临的问题。

为了建立一个条件随机场，我们首先要定义一个特征函数集，该函数集内的每个特征函数都以标注序列作为输入，提取的特征作为输出。假设该函数集为：



其中$x=(x_1, ..., x_m)$表示观测序列，$s = (s_1, ...., s_m)$表示状态序列。然后，条件随机场使用对数线性模型来计算给定观测序列下状态序列的条件概率：

![log_linear_crf](./imgs/log_linear_crf.png)

其中$s^{'}$是是所有可能的状态序列，$w$是条件随机场模型的参数，可以把它看成是每个特征函数的权重。CRF模型的训练其实就是对参数$w$的估计。假设我们有$n$个已经标注好的数据$\{(x^i, s^i)\}_{i=1}^n$，

则其对数似然函数的正则化形式如下：

![log_likehood_crf](./imgs/log_likehood_crf.png)

那么，最优参数$w^*$就是：

![w_crf](./imgs/w_crf.png)

模型训练结束之后，对给定的观测序列$x$，它对应的最优状态序列应该是：

![decode_crf](./imgs/decode_crf.png)

解码的时候与HMM类似，也可以采用维特比算法。

In [13]:
from sklearn_crfsuite import CRF

def word2features(sent, i):
    """抽取单个字的特征"""
    pass

    
def sent2features(sent):
    """抽取序列特征"""
    pass

class CRFModel(object):
    def __init__(self,
                 algorithm='lbfgs',
                 c1=0.1,
                 c2=0.1,
                 max_iterations=100,
                 all_possible_transitions=False
                 ):

        self.model = CRF(algorithm=algorithm,
                         c1=c1,
                         c2=c2,
                         max_iterations=max_iterations,
                         all_possible_transitions=all_possible_transitions)
    def train(self, sentences, tag_lists):
        # sentences是输入文本，tag_lists是标签
        #提示：self.model.fit(input, labels)函数可以用于训练
        pass

    def test(self, sentences):
        # sentences是输入文本
        #提示：self.model.predict(input)函数可以用于预测
        pass

In [5]:
def crf_train_eval(train_data, test_data, remove_O=False):

    # 训练CRF模型  由于CRF模型训练时间较长，这里给出训练好的模型，直接进行测试
    train_word_lists, train_tag_lists = train_data
    test_word_lists, test_tag_lists = test_data

    crf_model = CRFModel()
    crf_model.train(train_word_lists, train_tag_lists)
    
    save_model(crf_model, "./ckpts/crf.pkl")

    crf_model = load_model("./ckpts/crf.pkl")
    pred_tag_lists = crf_model.test(test_word_lists)

    metrics = Metrics(test_tag_lists, pred_tag_lists, remove_O=remove_O)
    metrics.report_scores()
    metrics.report_confusion_matrix()

    return pred_tag_lists, test_tag_lists

print("正在训练评估CRF模型...")
crf_pred = crf_train_eval(
    (train_word_lists, train_tag_lists),
    (test_word_lists, test_tag_lists)
)